<a href="https://colab.research.google.com/github/dilanbakr/chatbot/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

In [ ]:
from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Activation, Dense, LSTM, Permute, Dropout, BatchNormalization, add, dot, concatenate
from keras.utils.data_utils import get_file
from keras.preprocessing.sequence import pad_sequences
from keras.optimizers import Adam
from functools import reduce
import tarfile
import numpy as np
import re

In [ ]:
def tokenize(sent): ## splitting the text in tokens including punctuation
    return [x.strip() for x in re.split('(\W+)?', sent) if x.strip()]

In [ ]:
 a = tokenize("  jjhb ıhbjb, şojışk knk. ")
 a

/usr/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


['jjhb', 'ıhbjb', ',', 'şojışk', 'knk', '.']

In [ ]:
#Parse stories provided in the bAbi tasks format
#If only_supporting is true, only the sentences
#that support the answer are kept.
def parse_stories(lines, only_supporting=False):
    data = []
    story = []
    for line in lines:
        line = line.decode('utf-8').strip()
        id, line = line.split(' ', 1)
        id = int(id)
        if id == 1:
            story = []
        if '\t' in line:
            q, a, supporting = line.split('\t')
            q = tokenize(q)
            substory = None
            if only_supporting:
                # Only select the related substory
                supporting = map(int, supporting.split())
                substory = [story[i - 1] for i in supporting]
            else:
                # Provide all the substories
                substory = [x for x in story if x]
            data.append((substory, q, a))
            story.append('')
        else:
            sent = tokenize(line)
            story.append(sent)
    return data

In [ ]:
#Given a file, read the file, retrieve the stories,
#and then convert the sentences into a single story.
#If max_length is supplied, any stories longer than max_length tokens will be discarded.
def get_stories(file, only_supporting=False, max_length=None):
    data = parse_stories(file.readlines(), only_supporting=only_supporting)
    flat = lambda data: reduce(lambda i, j: i + j, data)
    data = [(flat(story), question, answer) for story, question, answer in data if not max_length or len(flat(story)) < max_length]
    return data

In [ ]:
#Given a file, read the file, retrieve the stories,
#and then convert the sentences into a single story.
#If max_length is supplied, any stories longer than max_length tokens will be discarded.
def get_stories(file, only_supporting=False, max_length=None):
    data = parse_stories(file.readlines(), only_supporting=only_supporting)
    flat = lambda data: reduce(lambda i, j: i + j, data)
    data = [(flat(story), question, answer) for story, question, answer in data if not max_length or len(flat(story)) < max_length]
    return data

In [ ]:
def vectorize_stories(data, word_id, story_maxlen, question_maxlen):
    X = []
    Q = []
    Y = []
    for story, question, answer in data:
        x = [word_id[i] for i in story]
        q = [word_id[i] for i in question]
        # Index 0 is reserved
        y = np.zeros(len(word_id) + 1)
        y[word_id[answer]] = 1
        X.append(x)
        Q.append(q)
        Y.append(y)
    return (pad_sequences(X, maxlen=story_maxlen), pad_sequences(Q, maxlen=question_maxlen), np.array(Y))

In [ ]:
tar_file = tarfile.open(get_file('babi-tasks-v1-2.tar.gz',
                                 origin='https://s3.amazonaws.com/text-datasets/babi_tasks_1-20_v1-2.tar.gz'))

11747328/11745123 [==============================] - 2s 0us/step


In [ ]:
challenges = {
    'single_supporting_fact_10k': 'tasks_1-20_v1-2/en-10k/qa1_single-supporting-fact_{}.txt', # QA1 with 10,000 samples
    'two_supporting_facts_10k': 'tasks_1-20_v1-2/en-10k/qa2_two-supporting-facts_{}.txt', # QA2 with 10,000 samples
}
challenge_type = 'single_supporting_fact_10k'
challenge = challenges[challenge_type]

train_stories = get_stories(tar_file.extractfile(challenge.format('train')))
test_stories = get_stories(tar_file.extractfile(challenge.format('test')))

/usr/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [ ]:
print('Train stories lenght:', len(train_stories))
print('Test stories lenght:', len(test_stories))

Train stories lenght: 10000
Test stories lenght: 1000


In [ ]:
train_stories[1]

(['Mary',
  'moved',
  'to',
  'the',
  'bathroom',
  '.',
  'John',
  'went',
  'to',
  'the',
  'hallway',
  '.',
  'Daniel',
  'went',
  'back',
  'to',
  'the',
  'hallway',
  '.',
  'Sandra',
  'moved',
  'to',
  'the',
  'garden',
  '.'],
 ['Where', 'is', 'Daniel', '?'],
 'hallway')

In [ ]:
test_stories[1]

(['John',
  'travelled',
  'to',
  'the',
  'hallway',
  '.',
  'Mary',
  'journeyed',
  'to',
  'the',
  'bathroom',
  '.',
  'Daniel',
  'went',
  'back',
  'to',
  'the',
  'bathroom',
  '.',
  'John',
  'moved',
  'to',
  'the',
  'bedroom',
  '.'],
 ['Where', 'is', 'Mary', '?'],
 'bathroom')

In [ ]:
vocab = set()
for story, question, answer in train_stories + test_stories:
    vocab |= set(story + question + [answer])
vocab = sorted(vocab)
#print(vocab)

In [ ]:
# Reserve 0 for masking via pad_sequences
vocab_size = len(vocab) + 1
vocab_size

22

In [ ]:
story_maxlen = max(map(len, (s for s, _, _ in train_stories + test_stories)))
question_maxlen = max(map(len, (s for _, s, _ in train_stories + test_stories)))

In [ ]:
print('Story max length:', story_maxlen)
print('Question max length:', question_maxlen)

Story max length: 68
Question max length: 4


In [ ]:
word2id = dict((w, i + 1) for i, w in enumerate(vocab))
print(word2id)

{'.': 1, '?': 2, 'Daniel': 3, 'John': 4, 'Mary': 5, 'Sandra': 6, 'Where': 7, 'back': 8, 'bathroom': 9, 'bedroom': 10, 'garden': 11, 'hallway': 12, 'is': 13, 'journeyed': 14, 'kitchen': 15, 'moved': 16, 'office': 17, 'the': 18, 'to': 19, 'travelled': 20, 'went': 21}


In [ ]:
inputs_train, questions_train, answers_train = vectorize_stories(train_stories, word2id, story_maxlen, question_maxlen)

In [ ]:
inputs_test, questions_test, answers_test = vectorize_stories(test_stories, word2id, story_maxlen, question_maxlen)

In [ ]:
print('inputs_train shape:', inputs_train.shape)
print('inputs_test shape:', inputs_test.shape)

inputs_train shape: (10000, 68)
inputs_test shape: (1000, 68)


In [ ]:
print('queries_train shape:', questions_train.shape)
print('queries_test shape:', questions_test.shape)

queries_train shape: (10000, 4)
queries_test shape: (1000, 4)


In [ ]:
print('answers_train shape:', answers_train.shape)
print('answers_test shape:', answers_test.shape)

answers_train shape: (10000, 22)
answers_test shape: (1000, 22)


In [ ]:
story_sequence = Input((story_maxlen,))
question = Input((question_maxlen,))

# embed the input sequence into a sequence of vectors for the stories
input_encoder_s = Sequential()
input_encoder_s.add(Embedding(input_dim=vocab_size, output_dim=64))
input_encoder_s.add(Dropout(0.3))

# embed the input into a sequence of vectors of size question_maxlen
# output: (samples, story_maxlen, question_maxlen)
input_encoder_q = Sequential()
input_encoder_q.add(Embedding(input_dim=vocab_size, output_dim=question_maxlen))
input_encoder_q.add(Dropout(0.3))

# embed the question into a sequence of vectors
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_size, output_dim=64, input_length=question_maxlen))
question_encoder.add(Dropout(0.3))

In [ ]:
# encode input sequence and questions to sequences of dense vectors
input_encoded_s = input_encoder_s(story_sequence)
input_encoded_q = input_encoder_q(story_sequence)
question_encoded = question_encoder(question)

# compute a 'match' between the first input vector sequence
# and the question vector sequence
# shape: `(samples, story_maxlen, question_maxlen)`
match = dot([input_encoded_s, question_encoded], axes=(2, 2))
match = Activation('softmax')(match)

# add the match matrix with the second input vector sequence
response = add([match, input_encoded_q])  # (samples, story_maxlen, question_maxlen)
response = Permute((2, 1))(response)  # (samples, question_maxlen, story_maxlen)

# concatenate the match matrix with the question vector sequence
answer = concatenate([response, question_encoded])

answer = LSTM(32)(answer)  # (samples, 32)
answer = Dropout(0.3)(answer)
answer = BatchNormalization()(answer)

output = Dense(vocab_size)(answer)  # (samples, vocab_size)
output = Activation('softmax')(output)

In [ ]:
model = Model([story_sequence, question], output)

In [ ]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 68)]         0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 4)]          0                                            
__________________________________________________________________________________________________
sequential_3 (Sequential)       (None, None, 64)     1408        input_3[0][0]                    
__________________________________________________________________________________________________
sequential_5 (Sequential)       (None, 4, 64)        1408        input_4[0][0]                    
_______________________________________________________________________________________

In [ ]:
model.compile(optimizer=Adam(0.005), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit([inputs_train, questions_train], answers_train,
          batch_size=128,
          epochs=120,
          validation_data=([inputs_test, questions_test], answers_test))

In [ ]:
pred = model.predict(([inputs_test, questions_test]))

In [ ]:
n = np.random.randint(0,1000)

In [ ]:
story_list = test_stories[n][0]
story =' '.join(word for word in story_list)
print("Story is:",story)

Story is: Daniel went to the bathroom . John went to the garden . John went back to the bedroom . Mary journeyed to the office . Daniel moved to the kitchen . John journeyed to the office . Mary journeyed to the kitchen . Sandra travelled to the bathroom .


In [ ]:
question_list = test_stories[n][1]
question =' '.join(word for word in question_list)
print("Question is: ", question)

Question is:  Where is John ?


In [ ]:
answer = test_stories[n][2]
print("Actual answer is: ", answer)

Actual answer is:  office


In [ ]:
max_value = np.argmax(pred[n])

In [ ]:
for key, val in word2id.items():
    if val == max_value:
        k = key
h
print("Machine answer is: ", k)
print("Machine says: I am ", pred[n][max_value], "certain of it")

Machine answer is:  the
Machine says: I am  0.05471424 certain of it
